# Gerar comparação entre o contexto e as palavras das sentenças do SQUAD2 P(1,0%) pt-br

Gera a comparação das palavras das sentenças com o contexto do documento do conjunto de dados SQUAD2 P(1,0%) pt-br utilizando os arquivos:
- `original.zip`
- `originalpos.zip`
- `perturbado_pX_kY.zip`
- `perturbadopos_pX_kY.zip`

Nos nomes dos arquivos `perturbado_pX_kY.zip`,`perturbadopos_pX_kY.zip`, X é o número de documentos perturbados e Y o valor de top K predições.

Cria o arquivo `comparacao_palavra1_pX_kY.zip` com as comparações entre as palavras do documento, onde X é o número de documentos perturbados e Y o valor de top K predições.

O arquivo é unido no segundo notebook.



# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.
    """

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))

    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [ ]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.

      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)

    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download
    nome_arquivo = nome_arquivo_destino.split("/")[-1]

    # Define o nome e caminho do arquivo temporário
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"

    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))

    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:
        tamanho_conteudo = data.headers.get("Content-Length")
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):
            if chunk:
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)

    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)

    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """

    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [ ]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "

    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [ ]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    sentencas = []
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())

    # Fecha o arquivo
    arquivo.close()

    return sentencas

In [ ]:
def salvar(nome_arquivo,texto):
    """
      Salva um texto em arquivo.

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Classe(ModelArguments) de definição dos parâmetros do modelo

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos a serem perturbados a partir do original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de palavras a serem recuperadas mais próximas da máscara."},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.

    Retorno:
    `device` - Um device de GPU ou CPU.
    """

    # Se existe GPU disponível.
    if torch.cuda.is_available():

        # Diz ao PyTorch para usar GPU.
        device = torch.device("cuda")

        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")

    return device

In [ ]:
device = getDeviceGPU()

Conecta o modelo ao device

In [ ]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.
        `device` - Um device de GPU.

      Retorno:
        `model` - Um objeto model BERT conectado a GPU.
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()

    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [ ]:
# Import das bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# Import das bibliotecas.
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

MessageError: ignored

## 1.7 Instalação do wandb

Instalação

In [ ]:
!pip install --upgrade wandb

## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [ ]:
# Instala o spacy
!pip install -U pip setuptools wheel

In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

## 1.9 Instalação do BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face.

In [ ]:
!pip install -U transformers==4.5.1

# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 20

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 20

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosMedida(
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
    versao_modelo_spacy = "3.2.0",
    sentenciar_documento = False,
    do_lower_case = False, # default True
    output_attentions = False, # default False
    output_hidden_states = True, # default False, se True retorna todas as camadas do modelo para as operações de soma e concatenação
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path.
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "SQUAD2_P"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [ ]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"

    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [ ]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
    """

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy

    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"

    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")
    arquivo_tar.extractall(DIRETORIO_COHEBERT)
    arquivo_tar.close()

    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.

    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp

### Carrega o modelo spaCy


In [ ]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [ ]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).

      Parâmetros:
        `nlp` - Um modelo spaCy carregado.
    """

    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords

Lista dos stopwords

In [ ]:
print("Quantidade de stopwords:", len(getStopwords(nlp)))

print(getStopwords(nlp))

### getVerbos
Localiza os verbos da sentença

In [ ]:
# Import das bibliotecas.
import spacy
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ]

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ]

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}},
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar
               ]

matcherv = Matcher(nlp.vocab)

matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):
  #Processa o período
  doc1 = nlp(periodo.text)

  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))

  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novo_dic = dict()

  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novo_dic[classe_gramatical] = qtde

  return novo_dic

In [ ]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novo_dic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}

  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novo_dic[classe_gramatical] = qtde

  return novo_dic

### getDicTodasPOSQtde

Conta as POS Tagging de uma sentença

In [ ]:
def getDicTodasPOSQtde(lista):

  # Dicionário com as tags e quantidades
  conjunto = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ": 0}

  for x in lista:
    valor = conjunto.get(x)
    if valor != None:
      conjunto[x] = valor + 1
    else:
      conjunto[x] = 1

  return conjunto

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [ ]:
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):

  # Soma os dicionários da lista
  novo_dic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))

  return novo_dic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [ ]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [ ]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [ ]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista_tokens = []
  lista_pos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:
    lista_tokens.append(token.text)
    lista_pos.append(token.pos_)

  return lista_tokens, lista_pos

### Tradução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [ ]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}

#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA"
  return traduzido

### getSentencaSemStopWord

Retorna uma lista dos tokens sem as stopwords.

In [ ]:
def getSentencaSemStopWord(sentenca, stopwords):

  # Lista dos tokens
  lista = []

  # Percorre os tokens da sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é uma stopword
    if token.lower() not in stopwords:
      lista.append(token)

  # Retorna o documento
  return lista

### getSentencaSalientePOS

Retorna uma lista das palavras do tipo especificado.

In [ ]:
def getSentencaSalientePOS(sentenca, pos, tipoSaliente="NOUN"):

  # Lista dos tokens
  lista = []

  # Percorre a sentença
  for i, token in enumerate(sentenca):

    # Verifica se o token é do tipo especeficado
    if pos[i] == tipoSaliente:
      lista.append(token)

  # Retorna o documento
  return lista

###removeStopWords

Remove as stopwords de um documento ou senteça.

In [ ]:
def removeStopWord(documento, stopwords):

  # Remoção das stopwords do documento
  documento_sem_stopwords = [palavra for palavra in documento.split() if palavra.lower() not in stopwords]

  # Concatena o documento sem os stopwords
  documento_limpo = " ".join(documento_sem_stopwords)

  # Retorna o documento
  return documento_limpo

### retornaRelevante

Retorna somente os palavras do documento ou sentença do tipo especificado.

In [ ]:
def retornaRelevante(documento, classe_relevante="NOUN"):

  # Corrigir!
  # Utilizar o documento já tokenizado pelo spacy!!!!
  # Existe uma lista com o documento e a sentença tokenizada pelo spacy

  # Realiza o parsing no spacy
  doc = nlp(documento)

  # Retorna a lista das palavras relevantes
  documento_com_substantivos = []
  for token in doc:
    #print("token:", token.pos_)
    if token.pos_ == classe_relevante:
      documento_com_substantivos.append(token.text)

  # Concatena o documento com os substantivos
  documento_concatenado = " ".join(documento_com_substantivos)

  # Retorna o documento
  return documento_concatenado

# 4 BERT

## 4.1 Modelo Pré-treinado BERT

### Funções Auxiliares

In [ ]:
def getNomeModeloBERT(model_args):
    '''
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"

    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"
    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"
        else:
            if 'bert' in model_args.pretrained_model_name_or_path:
                MODELO_BERT = "_BERT"

    return MODELO_BERT

In [ ]:
def getTamanhoBERT(model_args):
    '''
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"

    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"

    return TAMANHO_BERT

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """

    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO

        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"

        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo!")
            # Apaga o diretório e os arquivos existentes
            shutil.rmtree(DIRETORIO_MODELO)

        # Realiza o download do arquivo do modelo
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.
        arquivo_zip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivo_zip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)

        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta!".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada!")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO)

    logging.info("Modelo BERT ajustado copiado!")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """
    Verifica de onde utilizar o modelo.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """

    DIRETORIO_MODELO = None

    if model_args.usar_mcl_ajustado == True:
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()

        logging.info("Usando modelo BERT ajustado.")

    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")

    return DIRETORIO_MODELO

## 4.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).



In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None

    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 4.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

### Função carrega modelo BERT medida

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.


In [ ]:
# Import das bibliotecas.
from transformers import BertModel # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.

      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertModel.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertModel.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:
        `model` - Um objeto do modelo BERT carregado.
        `tokenizer` - Um objeto tokenizador BERT carregado.
    """

    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)

    # Variável para conter o modelo
    model = None

    # Carrega o modelo para cálculo da medida
    model = carregaModeloMedida(DIRETORIO_MODELO, model_args)

    # Carrega o tokenizador.
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)

    return model, tokenizer

### Carrega o BERT

In [ ]:
# Carrega o modelo e tokenizador do BERT
model, tokenizer = carregaBERT(model_args)

### Recupera detalhes do BERT

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = getNomeModeloBERT(model_args)

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = getTamanhoBERT(model_args)

## 4.4 Funções auxiliares do BERT

### concatenaListas

In [ ]:
def concatenaListas(lista, pos=1):
  lista_concat = []

  for x in lista:
      lista_concat = lista_concat + x[pos]

  return lista_concat

### getEmbeddingsCamadas

Funções que recuperam os embeddings das camadas:
- Primeira camada;
- Penúltima camada;
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingPrimeiraCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado = output[2][0]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  return resultado

def getEmbeddingPenultimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado = output[2][-2]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  return resultado

def getEmbeddingUltimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado = output[2][-1]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  return resultado

def getEmbeddingSoma4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  embedding_camadas = output[2][-4:]
  # Saída: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  # Usa o método `stack` para criar uma nova dimensão no tensor
  # com a concateção dos tensores dos embeddings.
  #Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado_stack = torch.stack(embedding_camadas, dim=0)
  # Saída: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>

  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultado_stack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>

  return resultado

def getEmbeddingConcat4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Cria uma lista com os tensores a serem concatenados
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)
  # Lista com os tensores a serem concatenados
  lista_concat = []

  # Percorre os 4 últimos
  for i in [-1,-2,-3,-4]:
      # Concatena da lista
      lista_concat.append(output[2][i])

  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)
  # Realiza a concatenação dos embeddings de todos as camadas
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> x <768 ou 1024>)
  resultado = torch.cat(lista_concat, dim=-1)

  # Saída: Entrada: (<1(lote)> x <qtde_tokens> x <3072 ou 4096>)
  return resultado

def getEmbeddingSomaTodasAsCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.

  # Retorna todas as camadas descontando a primeira(0)
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  embedding_camadas = output[2][1:]
  # Saída: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)

  # Usa o método `stack` para criar uma nova dimensão no tensor
  # com a concateção dos tensores dos embeddings.
  #Entrada: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)
  resultado_stack = torch.stack(embedding_camadas, dim=0)
  # Saída: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>

  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultado_stack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>

  return resultado

### getEmbeddingsVisual

Função para gerar as coordenadas de plotagem a partir das sentenças de embeddings.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsVisualUltimaCamada(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualSoma4UltimasCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualConcat4UltimasCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

In [ ]:
def getEmbeddingsVisualSomaTodasAsCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, documento_tokenizado

### getEmbeddings

Função para gerar os embeddings de sentenças.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [ ]:
def getEmbeddingsUltimaCamada(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsSoma4UltimasCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsConcat4UltimasCamadas(documento, modelo, tokenizer):
    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

In [ ]:
def getEmbeddingsSomaTodasAsCamadas(documento, modelo, tokenizer):

    # Adiciona os tokens especiais
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Divide a sentença em tokens
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário
    tokens_indexados = tokenizer.convert_tokens_to_ids(documento_tokenizado)

    # Marca cada um dos tokens como pertencentes à sentença "1".
    mascara_atencao = [1] * len(documento_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])

    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():
        # Retorno de model quando ´output_hidden_states=True´ é setado:
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    return token_embeddings, documento_tokenizado

### getEmbeddingsDocumento

Recupera os embeddings e tokens do documento sem buffer.

In [ ]:
def getEmbeddingsDocumento(documento, modelo, tokenizer):

    return getEmbeddingsConcat4UltimasCamadas(documento, modelo, tokenizer)

### getEmbeddingsDocumentoBuffer

Recupera os embeddings e tokens do documento com buffer.

In [ ]:
buffer_token_embeddings = {}

def getEmbeddingsDocumentoBuffer(documento, modelo, tokenizer):

    # Se documento está no dicionário retorna o embedding e os tokens
    if documento in buffer_token_embeddings:
        registro_buffer = buffer_token_embeddings.get(documento)
        return registro_buffer[0], registro_buffer[1]
    else:
        # Gera o embedding
        token_embeddings, documento_tokenizado = getEmbeddingsDocumento(documento, modelo, tokenizer)
        buffer_token_embeddings.update({documento: [token_embeddings, documento_tokenizado]})

        return  token_embeddings, documento_tokenizado

In [ ]:
def limpaBufferEmbedding():
    buffer_token_embeddings.clear()

### getDocumentoTokenizado

Retorna o documento tokenizado

In [ ]:
def getDocumentoTokenizado(documento, tokenizer):
    """
      Retorna o documento tokenizado pelo BERT.

      Parâmetros:
      `documento` - Documento a ser tokenizado.
      `tokenizer` - Tokenizador do BERT.
    """

    # Adiciona os tokens especiais.
    documento_marcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documento_tokenizado = tokenizer.tokenize(documento_marcado)

    del tokenizer

    return documento_tokenizado

### encontrarIndiceSubLista

Retorna os índices de início e fim da sublista na lista

In [ ]:
def encontrarIndiceSubLista(lista: List, sublista: List):
    """
    Localiza os índices de início e fim de uma sublista em uma lista.
    Baseado no algoritmo de https://codereview.stackexchange.com/questions/19627/finding-sub-list
    de  https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

    Parâmetros:
      `lista` - Uma lista.
      `sublista` - Uma sublista a ser localizada na lista.

    Retorno:
      Os índices de início e fim da sublista na lista.
    """
    # Tamanho da lista
    h = len(lista)
    # Tamanho da sblista
    n = len(sublista)
    # Cria um dicionário com os saltos descrescentes dos elementos n-1 da sublista
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    # Percorre a lista
    while i < h:
        # Percorre a sublista
        for j in range(n):
            # Se elemento da lista diferente da sublista pula a interação
            if lista[i - j] != sublista[-j - 1]:
              # Passa para o próximo elemento da lista saltando a sublista
              i += skip.get(lista[i], n)
              # Interrompe o for.
              break
        else:
            #Finalizando a pesquisa depois de executar todo o for(sem break)
            indice_inicio = i - n + 1
            indice_fim = indice_inicio + len(sublista)-1

            # Retorna o início e fim da sublista na lista
            return indice_inicio, indice_fim

    # Não encontrou a sublista na lista
    return -1, -1

### getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras

A partir dos embeddings do documento, localiza o indíce de início e fim de uma sentença no documento e retorna os embeddings da sentença.

In [ ]:
def getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras(embedding_documento,
                                                           token_BERT_documento,
                                                           sentenca,
                                                           tokenizer):

  # Tokeniza a sentença
  sentenca_tokenizada_BERT = getDocumentoTokenizado(sentenca, tokenizer)
  #print(sentenca_tokenizada_BERT)

  # Remove os tokens de início e fim da sentença
  sentenca_tokenizada_BERT.remove("[CLS]")
  sentenca_tokenizada_BERT.remove("[SEP]")
  #print(len(sentenca_tokenizada_BERT))

  # Localiza os índices dos tokens da sentença no documento
  inicio, fim = encontrarIndiceSubLista(token_BERT_documento, sentenca_tokenizada_BERT)
  #print(inicio,fim)

  # Recupera os embeddings dos tokens da sentença a partir dos embeddings do documento
  embedding_sentenca = embedding_documento[inicio:fim+1]
  #print("embedding_sentenca=", embedding_sentenca.shape)

  del embedding_documento
  del token_BERT_documento
  del sentenca
  del tokenizer

  # Retorna o embedding da sentença no documento
  return embedding_sentenca, sentenca_tokenizada_BERT

### getEmbeddingDocumentoComTodasPalavrasMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoComTodasPalavrasMean(embedding_documento):
  """
    Calcula a média dos embeddings do documento excluindo os tokens
    especiais [CLS] do início e [SEP] do fim.
    Remove primeira dimensão devido ao cálculo da média.

    Parâmetros:
    `embedding_documento` - Embedding do documento.
  """

  # Calcula a média dos embeddings para os tokens de embedding_documento, removendo a primeira dimensão.
  # Entrada: <qtde_tokens> x <768 ou 1024>
  #print("embedding_documento1=", embedding_documento.shape)
  media_embedding_documento = torch.mean(embedding_documento[1:-1], dim=0)
  # Saída: <768 ou 1024>

  del embedding_documento

  return media_embedding_documento

### getEmbeddingDocumentoSemStopWordMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoSemStopWordMean(id_documento,
                                         index_sentenca,
                                         embedding_documento,
                                         token_BERT_documento,
                                         documento,
                                         tokenizer,
                                         token_documento,
                                         postagging_documento):
  """
    Calcula a média dos embeddings do documento considerando tokens do tipo
    especificado no filtro
    Remove primeira dimensão devido ao cálculo da média.

    Parâmetros:
    `embedding_documento` - Embeddings do documento gerados pelo BERT.
    `token_BERT_documento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `postagging_documento` - Lista com as POS-Tagging do documento.
  """

  # Recupera a lista de tokens do documento, a lista dos postagging e a lista dos seus embeddings com um mesmo tamanho
  lista_tokens, lista_postagging, lista_embeddings_mean, lista_embeddings_max = getTokensEmbeddingsPOSSentenca(id_documento,
                                                                                                               index_sentenca,
                                                                                                               embedding_documento,
                                                                                                               token_BERT_documento,
                                                                                                               documento,
                                                                                                               tokenizer,
                                                                                                               token_documento,
                                                                                                               postagging_documento)

  #print("len(token_BERT_documento):", len(token_BERT_documento))
  #print("token_BERT_documento:", token_BERT_documento)
  #print("len(postagging_documento):", len(postagging_documento))
  #print("postagging_documento:", postagging_documento)
  #print("filtro:", filtro)
  #print()
  # Lista com os tensores selecionados
  lista_tokens_selecionados_mean = []

  # Localizar os embeddings dos tokens da sentença tokenizada sem stop word no documento
  for i, token_documento in enumerate(lista_tokens):
      if lista_tokens[i].lower() not in getStopwords(nlp):
          #print("Adicionando palavra do embedding:", lista_tokens[i])
          lista_tokens_selecionados_mean.append(lista_embeddings_mean[i])

  if  len(lista_tokens_selecionados_mean) != 0:
      # Empilha os embeddings da lista pela dimensão 0
      embedding_sem_stopword = torch.stack(lista_tokens_selecionados_mean, dim=0)
      #print("embedding_sem_stopword.shape:",embedding_sem_stopword.shape)

      # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
      # Entrada: <qtde_tokens> x <768 ou 1024>
      media_embedding_sem_stopword = torch.mean(embedding_sem_stopword, dim=0)
      # Saída: <768 ou 1024>
      #print("media_embedding_sem_stopword.shape:", media_embedding_sem_stopword.shape)
  else:
      media_embedding_sem_stopword = None

  del embedding_documento
  del token_BERT_documento
  del documento
  del tokenizer
  del token_documento
  del postagging_documento

  return media_embedding_sem_stopword

### getEmbeddingDocumentoRelevanteMean

In [ ]:
# Importa a biblioteca
import torch

def getEmbeddingDocumentoRelevanteMean(id_documento,
                                       index_sentenca,
                                       embedding_documento,
                                       token_BERT_documento,
                                       documento,
                                       tokenizer,
                                       token_documento,
                                       postagging_documento,
                                       filtro):
  """
    Calcula a média dos embeddings do documento considerando tokens do tipo
    especificado no filtro
    Remove primeira dimensão devido ao cálculo da média.

    Parâmetros:
    `embedding_documento` - Embeddings do documento gerados pelo BERT.
    `token_BERT_documento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `postagging_documento` - Lista com as POS-Tagging do documento.
    `filtro` - Filtro dos embeddings.

  """

  # Recupera a lista de tokens do documento, a lista dos postagging e a lista dos seus embeddings com um mesmo tamanho
  lista_tokens, lista_postagging, lista_embeddings_mean, lista_embeddings_max = getTokensEmbeddingsPOSSentenca(id_documento,
                                                                                                               index_sentenca,
                                                                                                               embedding_documento,
                                                                                                               token_BERT_documento,
                                                                                                               documento,
                                                                                                               tokenizer,
                                                                                                               token_documento,
                                                                                                               postagging_documento)

  #print("len(token_BERT_documento):", len(token_BERT_documento))
  #print("token_BERT_documento:", token_BERT_documento)
  #print("len(postagging_documento):", len(postagging_documento))
  #print("postagging_documento:", postagging_documento)
  #print("filtro:", filtro)
  #print()

  # Lista com os tensores selecionados
  lista_tokens_selecionados = []
  # Localizar os embeddings dos tokens da sentença tokenizada sem stop word no documento
  for i, token_documento in enumerate(lista_tokens):
    if (lista_postagging[i] in filtro):
      #print("Adicionando palavra do embedding:", lista_tokens[i])
      lista_tokens_selecionados.append(lista_embeddings_mean[i])

  if len(lista_tokens_selecionados) != 0:
      # Empila os embeddings da lista pela dimensão 0
      embedding_relevante = torch.stack(lista_tokens_selecionados, dim=0)
      #print("embedding_relevante.shape:",embedding_relevante.shape)

      # Calcula a média dos embeddings para os tokens de Si, removendo a primeira dimensão.
      # Entrada: <qtde_tokens> x <768 ou 1024>
      media_embedding_relevante = torch.mean(embedding_relevante, dim=0)
      # Saída: <768 ou 1024>
      #print("media_embedding_relevante.shape:", media_embedding_relevante.shape)
  else:
      media_embedding_relevante = None

  del embedding_documento
  del token_BERT_documento
  del documento
  del tokenizer
  del token_documento
  del postagging_documento

  return media_embedding_relevante

### getEmbeddingDocumentoMean

Filtros:
- ALL - Sentença com todas as palavras
- CLEAN - Sentença com todas as palavras menos as stopwords
- VERB,AUX - Sentença somente com verbos(VERB e AUX)
- NOUN - Sentença somente com substantivos
- VERB,NOUN,AUX - Sentença somente com verbos(VERB e AUX) e substantivos

In [ ]:
def getEmbeddingDocumentoMean(id_documento,
                              index_sentenca,
                              embedding_documento,
                              token_BERT_documento,
                              documento,
                              tokenizer,
                              token_documento,
                              postagging_documento,
                              filtro=["ALL"]):
  """
    Rediciona o cálculo da média dos embeddings de acordo com o filtro especificado.

    Parâmetros:
    `embedding_documento` - Embeddings do documento gerados pelo BERT.
    `token_BERT_documento` - Lista com os tokens do documento gerados pelo tokenizador BERT.
    `documento` - Texto com o documento.
    `tokenizer` - Tokenizador do BERT.
    `token_documento` - Lista com os tokens do documento.
    `postagging_documento` - Lista com as POS-Tagging do documento.
    `filtro` - Filtro dos embeddings.
  """

  if "ALL" in filtro:
    return getEmbeddingDocumentoComTodasPalavrasMean(embedding_documento)
  else:
    if "CLEAN" in filtro:
      return getEmbeddingDocumentoSemStopWordMean(id_documento,
                                                  index_sentenca,
                                                  embedding_documento,
                                                  token_BERT_documento,
                                                  documento,
                                                  tokenizer,
                                                  token_documento,
                                                  postagging_documento)
    else:
      return getEmbeddingDocumentoRelevanteMean(id_documento,
                                                index_sentenca,
                                                embedding_documento,
                                                token_BERT_documento,
                                                documento,
                                                tokenizer,
                                                token_documento,
                                                postagging_documento,
                                                filtro)

# 5 Comparar Contexto e Palavras

## 5.1 Carregamento dos arquivos de dados originais

### 5.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "originalpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

NOME_ARQUIVO_PERTURBADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_COMPACTADO = "perturbado_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"
NOME_ARQUIVO_PERTURBADO_POS = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO = "perturbadopos_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.1.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}.".format(dirbase))
else:
    logging.info("Diretório já existe: {}.".format(dirbase))

### 5.1.3 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens
*   `-d` Diretório de destino


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_PERTURBADO_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação!")

### 5.1.4 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS

In [ ]:
#Biblioteca
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

In [ ]:
lista_documentos_originais.sample(5)

In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str}

lista_documentos_originais = lista_documentos_originais.astype(tipos)

In [ ]:
lista_documentos_originais_pos.sample(5)

In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str}

lista_documentos_originais_pos = lista_documentos_originais_pos.astype(tipos)

#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

#### Criando dados indexados originais

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia dda lista inddexada
lista_documentos_originais_indexado = lista_documentos_originais.set_index(["id"])
lista_documentos_originais_indexado.head()

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia dda lista inddexada
lista_documentos_originais_pos_indexado = lista_documentos_originais_pos.set_index(["id"])
lista_documentos_originais_pos_indexado.head()

#### Carrega o arquivo dos dados perturbados e POS

In [ ]:
# Abre o arquivo e retorna o DataFrame
lista_documentos_perturbados = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO, sep=";", encoding="UTF-8")
lista_documentos_perturbados_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_PERTURBADO_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO PERTURBADOS: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO PERTURBADOS POS: {}.".format(len(lista_documentos_perturbados_pos)))

In [ ]:
lista_documentos_perturbados.sample(5)

In [ ]:
lista_documentos_perturbados_pos.sample(5)

#### Corrigir os tipos de colunas dos dados perturbados e POS

Em dados perturbados:
- coluna 1 - `perturbado` carregadas do arquivo vem como string e não como lista.
- coluna 3 - `sentencas` carregadas do arquivo vem como string e não como lista.

Em dados perturbados pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
lista_documentos_perturbados["perturbado"] = lista_documentos_perturbados["perturbado"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)
lista_documentos_perturbados["sentencas"] = lista_documentos_perturbados["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

lista_documentos_perturbados_pos["pos_documento"] = lista_documentos_perturbados_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO PERTURBADO: {}.".format(len(lista_documentos_perturbados)))
logging.info("TERMINADO CORREÇÃO PERTURBADO POS: {}.".format(len(lista_documentos_perturbados_pos)))

#### Criando dados indexados perturbados

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia dda lista inddexada
lista_documentos_perturbados_indexado = lista_documentos_perturbados.set_index(["id"])
lista_documentos_perturbados_indexado.head()

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia dda lista inddexada
lista_documentos_perturbados_pos_indexado = lista_documentos_perturbados_pos.set_index(["id"])
lista_documentos_perturbados_pos_indexado.head()

### 5.1.5 Agrupar os dados originais e perturbados

In [ ]:
# Import das bibliotecas.
import ast
from tqdm.notebook import tqdm as tqdm_notebook

print("Processando",len(lista_documentos_originais),"documentos originais")

lista_documentos_agrupados = []

# Barra de progresso dos documentos
lista_documentos_originais_bar = tqdm_notebook(lista_documentos_originais.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_originais))

# Percorre os documentos
for i, linha_documento in lista_documentos_originais_bar:
  #if i < 2:
    #print("linha_documento:",linha_documento)
    # Recupera o id do documento
    id_documento = linha_documento[0]
    #print("id_documento:",id_documento)

    # Carrega a lista das sentenças do documento
    lista_sentenca_documento = linha_documento[1]
    #print("\nlista_sentenca_documento:",lista_sentenca_documento)
    #print("len(lista_sentenca_documento):",len(lista_sentenca_documento))

    # Adiciona o original a lista dos dados agrupados, considerando como coerente(1)
    lista_documentos_agrupados.append([id_documento, lista_sentenca_documento, linha_documento[2], 1])

    # Percorre os documentos perturbados apartir do original
    for j in range(0, model_args.documentos_perturbados):

      # Id do documento perturbado
      id_perturbado = str(id_documento) + "_pert_" + str(j)

      # localiza o documento perturbado
      documento_perturbado = lista_documentos_perturbados_indexado.loc[id_perturbado]
      # Recupera a sentença do documento perturbado
      lista_perturbado = documento_perturbado[0]

      # Adiciona o perturbado a lista dos dados agrupados considerando como incoerente(0)
      lista_documentos_agrupados.append([id_perturbado, lista_perturbado, documento_perturbado[1], 0])

logging.info("TERMINADO AGRUPAMENTO: {}.".format(len(lista_documentos_agrupados)))

#### Converte a lista em um dataframe

In [ ]:
# Cria o dataframe da lista
lista_documentos_agrupados = pd.DataFrame(lista_documentos_agrupados, columns = ["id","sentencas","documento","classe"])

In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str, "sentencas": object, "documento": str, "classe": int}

lista_documentos_agrupados = lista_documentos_agrupados.astype(tipos)

In [ ]:
lista_documentos_agrupados.sample(5)

In [ ]:
# Importa das bibliotecas
import pandas as pd

# Concatena as listas de documentos originais e perturbados
lista_documentos_agrupados_pos = pd.concat([lista_documentos_originais_pos, lista_documentos_perturbados_pos])

logging.info("TERMINADO AGRUPAMENTO POS: {}.".format(len(lista_documentos_agrupados_pos)))

In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str}

lista_documentos_agrupados_pos = lista_documentos_agrupados_pos.astype(tipos)

In [ ]:
lista_documentos_agrupados_pos.sample(5)

#### Criar dados indexados

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia dda lista inddexada
lista_documentos_agrupados_indexado = lista_documentos_agrupados.set_index(["id"])
lista_documentos_agrupados_indexado.head()

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia dda lista inddexada
lista_documentos_agrupados_pos_indexado = lista_documentos_agrupados_pos.set_index(["id"])
lista_documentos_agrupados_pos_indexado.head()

## 5.2 Gerando as comparações



### 5.2.1 Medidas de similaridade


Similaridade do cosseno entre os embeddings.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine

A função spatial.distance.cosine do módulo scipy calcula a distância em vez da similaridade do cosseno, mas para conseguir isso, podemos subtrair o valor da distância de 1.

Intervalo de [-1,1]

Vetores iguais a distância é igual 1.

Vetores diferentes medida próxima de -1.

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cosine

def similaridadeCosseno(embeddings1, embeddings2):
    """
      Similaridade do cosseno dos embeddings dos textos.

      Parâmetros:
      `embeddings1` - Um embedding a ser medido.
      `embeddings2` - Um embedding a ser medido.
    """

    similaridade = 1 - cosine(embeddings1, embeddings2)

    return similaridade

### 5.2.2 Medidas de distância

Distância euclidiana entre os embeddings.

Possui outros nomes como distância L2 ou norma L2.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html#scipy.spatial.distance.euclidean

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import euclidean

def distanciaEuclidiana(embeddings1, embeddings2):
    """
    Distância euclidiana entre os embeddings dos textos.
    Possui outros nomes como distância L2 ou norma L2.

    Parâmetros:
    `embeddings1` - Um embedding a ser medido.
    `embeddings2` - Um embedding a ser medido.
    """

    distancia = euclidean(embeddings1, embeddings2)

    return distancia

Distância Manhattan entre os embeddings.

Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cityblock.html#scipy.spatial.distance.cityblock

In [ ]:
# Import das bibliotecas.
from scipy.spatial.distance import cityblock

def distanciaManhattan(embeddings1, embeddings2):
    """
    Distância Manhattan entre os embeddings dos textos
    Possui outros nomes como distância Cityblock, distância L1, norma L1 e métrica do táxi.

    Parâmetros:
    `embeddings1` - Um embedding a ser medido.
    `embeddings2` - Um embedding a ser medido.
    """

    distancia = cityblock(embeddings1, embeddings2)

    return distancia

### 5.2.3 Retorna as medidas dos embeddings

In [ ]:
def getMedidasEmbedding(embedding_wi, embedding_wj):

  """
    Retorna as medidas de similaridade do cosseno(cos), distância Euclidiana(euc) e
    distância de Manhattan(man) entre os embeddings.

    Parâmetros:
    `embeddings_wi` - Um embedding de uma palavra a ser medido.
    `embeddings_wj` - Um embedding de uma palavra a ser medido.
  """

  #print("embedding_wi=", embedding_wi.shape)
  #print("embedding_wj=", embedding_wj.shape)

  # Similaridade do cosseno entre os embeddings wi e wj
  # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
  cos = similaridadeCosseno(embedding_wi, embedding_wj)
  # Saída: Número real

  # Distância euclidiana entre os embeddings wi e wj
  # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
  euc = distanciaEuclidiana(embedding_wi, embedding_wj)
  # Saída: Número real

  # Distância de manhattan entre os embeddings wi e wj
  # Entrada: (<768 ou 1024>) x (<768 ou 1024>)
  man = distanciaManhattan(embedding_wi, embedding_wj)
  # Saída: Número real

  del embedding_wi
  del embedding_wj

  # Retorno das medidas das sentenças
  return cos, euc, man

### 5.2.4 getTokensEmbeddingsPOSSentenca
Gera os tokens, POS e embeddings de cada sentença.

In [ ]:
# Dicionário de tokens de exceções e seus deslocamentos para considerar mais tokens do BERT em relação ao spaCy
# A tokenização do BERT gera mais tokens que a tokenização das palavras do spaCy
dic_excecao_maior = {"al-Ḥarrānī": 3,
               "al-Battānī": 7,
               "mi.²":3,
               "nm":2,
               "550":2,
               "mg":1,
               "550nm":4,
               "q-glass":5,
               "ômega-6":3,
               str(chr(804)+chr(10217)):1,
               "mm":1,
               "K":1,
               "al-qasim":3,
               "T":1,
               "ḱlew-":2,
               "◌":1,
               "m":1,
               "TGF-β":5,
               "56.º":3,
               "45.º":3,
               "34.º":3,
               "nº":1,
               "200º":1,
               "1º":1,
               "g":1,
               "š":1,
               "ž":1,
               "km²":1,
               "κανὠν":1,
               "7º":1,
               "2º":1,
               "3º":1,
               "19º":1,
               "18º":1,
               "ʱ⟩":1,
               "ð":1,
               "θ":1,
               "Ü-Tsang":4,
               "p˭":1,
               "s˭":1,
               "pʰ":1,
               "ʰp":1,
               "sʰ":1,
               "q":1,
               "Ṣalībī":1,
               "ṣalīb":1,
               "⟨bʰ⟩":3,
               "Ônibus":1,
               "2ª":1,
               "indo-arianas":4,
               "G":1,
               "⟨bd":3,
               "ɡ⟩":2,
               "tʰ":1,}

In [ ]:
def getExcecaoDicMaior(id, token, dic_excecao_maior):

  valor = dic_excecao_maior.get(token)
  if valor != None:

    if token == "mg" and id in ["572b8803be1ee31400cb8405",
                                "572b8803be1ee31400cb8406",
                                "5acfcfd677cf76001a68614c",
                                "5acfcfd677cf76001a68614d"]:
      return 2
    else:
      return valor
  else:
    return -1

In [ ]:
# Dicionário de tokens de exceções e seus deslocamentos para considerar menos tokens do BERT em relação ao spaCy
# A tokenização do BERT gera menos tokens que a tokenização das palavras do spaCy
dic_excecao_menor = {"1°":1,
                    }

In [ ]:
def getExcecaoDicMenor(id, token, dic_excecao_menor):

  valor = dic_excecao_menor.get(token)
  if valor != None:
      return valor
  else:
      return -1

Função que retorna os embeddings, tokens e POS da sentença com um mesmo tamamnho.

In [ ]:
# Importa a biblioteca
import torch

def getTokensEmbeddingsPOSSentenca(id_documento,
                                   index_sentenca,
                                   embedding_documento,
                                   token_BERT_documento,
                                   sentenca,
                                   tokenizer,
                                   sentenca_token = None,
                                   sentenca_postagging = None,
                                   estrategia_medida = 0):
    """
      Retorna os tokens, as POS-Tagging e os embeddings dos tokens igualando a quantidade de tokens do spaCy com a tokenização do BERT de acordo com a estratégia de pooling para palavras fora do vocabulário do BERT.
      Usa a estratégia MEAN para calcular a média dos embeddings dos tokens que formam uma palavra fora do vocabulário do BERT.
      Usa a estratégia MAX para calcular o valor máximo dos embeddings dos tokens que formam uma palavra fora do vocabulário do BERT.
    """

    #Guarda os tokens e embeddings
    lista_tokens = []
    lista_embeddings_mean = []
    lista_embeddings_max = []

    # Se a sentença não for tokenizada
    if sentenca_token == None:
      # Gera a tokenização e POS-Tagging da sentença
      sentenca_token, sentenca_postagging = getListaTokensPOSSentenca(sentenca)

    #print("\nsentenca                :",sentenca)
    #print("id_documento                :",id_documento)
    #print("index_sentenca              :",index_sentenca)
    #print("sentenca_token              :",sentenca_token)
    #print("len(sentenca_token)         :",len(sentenca_token))
    #print("sentenca_postagging         :",sentenca_postagging)
    #print("len(sentenca_postagging)    :",len(sentenca_postagging))

    # Recupera os embeddings da sentença dos embeddings dentro dos embeddings do documento
    embedding_sentenca, sentenca_tokenizada_BERT = getEmbeddingSentencaEmbeddingDocumentoComTodasPalavras(embedding_documento,
                                                                                                       token_BERT_documento,
                                                                                                       sentenca,
                                                                                                       tokenizer)

    # embedding <qtde_tokens x 4096>
    #print("embedding_sentenca          :",embedding_sentenca.shape)
    #print("sentenca_tokenizada_BERT     :",sentenca_tokenizada_BERT)
    #print("len(sentenca_tokenizada_BERT):",len(sentenca_tokenizada_BERT))

    # Seleciona os pares de palavra a serem avaliadas
    pos_wi = 0 # Posição do token da palavra gerado pelo spaCy
    pos_wj = pos_wi # Posição do token da palavra gerado pelo BERT
    pos2 = -1

    # Enquanto o indíce da palavra pos_wj(2a palavra) não chegou ao final da quantidade de tokens do BERT
    while pos_wj < len(sentenca_tokenizada_BERT):

      # Seleciona os tokens da sentença
      wi = sentenca_token[pos_wi] # Recupera o token da palavra gerado pelo spaCy
      wi1 = ""
      pos2 = -1
      if pos_wi+1 < len(sentenca_token):
        wi1 = sentenca_token[pos_wi+1] # Recupera o próximo token da palavra gerado pelo spaCy

        # Localiza o deslocamento da exceção
        pos2 = getExcecaoDicMenor(id_documento, wi+wi1, dic_excecao_menor)
        #print("Exceção pos2:", pos2)

      wj = sentenca_tokenizada_BERT[pos_wj] # Recupera o token da palavra gerado pelo BERT
      #print("wi[",pos_wi,"]=", wi)
      #print("wj[",pos_wj,"]=", wj)

      # Tratando exceções
      # Localiza o deslocamento da exceção
      pos = getExcecaoDicMaior(id_documento, wi, dic_excecao_maior)
      #print("Exceção pos:", pos)

      if pos != -1 or pos2 != -1:
        if pos != -1:
          #print("Adiciona 1 Exceção palavra == wi or palavra = [UNK]:",wi)
          lista_tokens.append(wi)
          # Verifica se tem mais de um token
          if pos != 1:
            indice_token = pos_wj + pos
            #print("Calcula a média de :", pos_wj , "até", indice_token)
            embeddings_tokens_palavra = embedding_sentenca[pos_wj:indice_token]
            #print("embeddings_tokens_palavra:",embeddings_tokens_palavra.shape)
            # calcular a média dos embeddings dos tokens do BERT da palavra
            embedding_estrategia_mean = torch.mean(embeddings_tokens_palavra, dim=0)
            #print("embedding_estrategia_mean:",embedding_estrategia_mean.shape)
            lista_embeddings_mean.append(embedding_estrategia_mean)

            # calcular o máximo dos embeddings dos tokens do BERT da palavra
            embedding_estrategia_max, linha = torch.max(embeddings_tokens_palavra, dim=0)
            #print("embedding_estrategia_max:",embedding_estrategia_max.shape)
            lista_embeddings_max.append(embedding_estrategia_max)
          else:
            # Adiciona o embedding do token a lista de embeddings
            lista_embeddings_mean.append(embedding_sentenca[pos_wj])
            lista_embeddings_max.append(embedding_sentenca[pos_wj])

          # Avança para a próxima palavra e token do BERT
          pos_wi = pos_wi + 1
          pos_wj = pos_wj + pos
          #print("Proxima:")
          #print("wi[",pos_wi,"]=", sentenca_token[pos_wi])
          #print("wj[",pos_wj,"]=", sentenca_tokenizada_BERT[pos_wj])
        else:
          if pos2 != -1:
            #print("Adiciona 1 Exceção palavra == wi or palavra = [UNK]:",wi)
            lista_tokens.append(wi+wi1)
            # Verifica se tem mais de um token
            if pos2 == 1:
              # Adiciona o embedding do token a lista de embeddings
              lista_embeddings_mean.append(embedding_sentenca[pos_wj])
              lista_embeddings_max.append(embedding_sentenca[pos_wj])

            # Avança para a próxima palavra e token do BERT
            pos_wi = pos_wi + 2
            pos_wj = pos_wj + pos2
            #print("Proxima:")
            #print("wi[",pos_wi,"]=", sentenca_token[pos_wi])
            #print("wj[",pos_wj,"]=", sentenca_tokenizada_BERT[pos_wj])
      else:
        # Tokens iguais adiciona a lista, o token não possui subtoken
        if (wi == wj or wj=="[UNK]"):
          # Adiciona o token a lista de tokens
          #print("Adiciona 2 wi==wj or wj==[UNK]:", wi )
          lista_tokens.append(wi)
          # Adiciona o embedding do token a lista de embeddings
          lista_embeddings_mean.append(embedding_sentenca[pos_wj])
          lista_embeddings_max.append(embedding_sentenca[pos_wj])
          #print("embedding1[pos_wj]:", embedding_sentenca[pos_wj].shape)
          # Avança para a próxima palavra e token do BERT
          pos_wi = pos_wi + 1
          pos_wj = pos_wj + 1

        else:
          # A palavra foi tokenizada pelo Wordpice com ## ou diferente do spaCy ou desconhecida
          # Inicializa a palavra a ser montada
          palavra_postagging = wj
          indice_token = pos_wj + 1
          while  ((palavra_postagging != wi) and indice_token < len(sentenca_tokenizada_BERT)):
              if "##" in sentenca_tokenizada_BERT[indice_token]:
                # Remove os caracteres "##" do token
                parte = sentenca_tokenizada_BERT[indice_token][2:]
              else:
                parte = sentenca_tokenizada_BERT[indice_token]

              palavra_postagging = palavra_postagging + parte
              #print("palavra_postagging:",palavra_postagging)
              # Avança para o próximo token do BERT
              indice_token = indice_token + 1

          #print("\nMontei palavra:",palavra_postagging)
          if (palavra_postagging == wi or palavra_postagging == "[UNK]"):
              # Adiciona o token a lista
              #print("Adiciona 3 palavra == wi or palavra_postagging = [UNK]:",wi)
              lista_tokens.append(wi)
              # Calcula a média dos tokens da palavra
              #print("Calcula o máximo :", pos_wj , "até", indice_token)
              embeddings_tokens_palavra = embedding_sentenca[pos_wj:indice_token]
              #print("embeddings_tokens_palavra2:",embeddings_tokens_palavra)
              #print("embeddings_tokens_palavra2:",embeddings_tokens_palavra.shape)

              # calcular a média dos embeddings dos tokens do BERT da palavra
              embedding_estrategia_mean = torch.mean(embeddings_tokens_palavra, dim=0)
              #print("embedding_estrategia_mean:",embedding_estrategia_mean)
              #print("embedding_estrategia_mean.shape:",embedding_estrategia_mean.shape)
              lista_embeddings_mean.append(embedding_estrategia_mean)

              # calcular o valor máximo dos embeddings dos tokens do BERT da palavra
              embedding_estrategia_max, linha = torch.max(embeddings_tokens_palavra, dim=0)
              #print("embedding_estrategia_max:",embedding_estrategia_max)
              #print("embedding_estrategia_max.shape:",embedding_estrategia_max.shape)
              lista_embeddings_max.append(embedding_estrategia_max)

          # Avança para o próximo token do spaCy
          pos_wi = pos_wi + 1
          # Pula para o próximo token do BERT
          pos_wj = indice_token

    # Verificação se as listas estão com o mesmo tamanho
    #if (len(lista_tokens) != len(sentenca_token)) or (len(lista_embeddings_mean) != len(sentenca_token)):
    if (len(lista_tokens) !=  len(lista_embeddings_mean)):
       print("\nsentenca                  :",sentenca)
       print("id_documento              :",id_documento)
       print("index_sentenca            :",index_sentenca)
       print("sentenca_postagging       :",sentenca_postagging)
       print("sentenca_token            :",sentenca_token)
       print("sentenca_tokenizada_BERT  :",sentenca_tokenizada_BERT)
       print("lista_tokens              :",lista_tokens)
       print("len(lista_tokens)         :",len(lista_tokens))
       print("lista_embeddings_mean     :",lista_embeddings_mean)
       print("len(lista_embeddings_mean):",len(lista_embeddings_mean))
       print("lista_embeddings_max      :",lista_embeddings_max)
       print("len(lista_embeddings_max) :",len(lista_embeddings_max))

    del embedding_sentenca
    del token_BERT_documento
    del tokenizer
    del sentenca_tokenizada_BERT
    del sentenca_token

    return lista_tokens, sentenca_postagging, lista_embeddings_mean, lista_embeddings_max

### 5.2.5 comparaContextoPalavrasTodas

In [ ]:
def comparaContextoPalavrasTodas(id_documento,
                                 indexdocumento,
                                 index_sentenca,
                                 embedding_documento,
                                 token_BERT_documento,
                                 sentenca,
                                 tokenizer,
                                 sentenca_token,
                                 sentenca_postagging,
                                 contextos_documentos):

  lista_tokens, lista_postagging, lista_embeddings_mean, lista_embeddings_max = getTokensEmbeddingsPOSSentenca(id_documento,
                                                                                                               index_sentenca,
                                                                                                               embedding_documento,
                                                                                                               token_BERT_documento,
                                                                                                               sentenca,
                                                                                                               tokenizer,
                                                                                                               sentenca_token,
                                                                                                               sentenca_postagging)

  #print("\nSentença   :",lista_tokens)
  #print("POS Tagging:",lista_postagging)
  #print("Quantidade de palavras:",len(lista_tokens))

  # Quantidade de palavras no documento
  k = len(lista_tokens)

  # Guarda a comparação da sentença
  lista_comparacao = []

  # Percorre as palavras da sentença
  for i in range(0,k):
    # Seleciona a palavra i da sentença
    wi = lista_tokens[i]
    posi = lista_postagging[i]

    # Contexto de todas as palavras
    if contextos_documentos[0] != None:
      # Recupera as medidas dos embeddings das palavras usando as estratégias MEAN e MAX
      cos_ctxall_mean, euc_ctxall_mean, man_ctxall_mean = getMedidasEmbedding(lista_embeddings_mean[i],
                                                                        contextos_documentos[0])
      cos_ctxall_max, euc_ctxall_max, man_ctxall_max = getMedidasEmbedding(lista_embeddings_max[i],
                                                                     contextos_documentos[0])
    else:
      cos_ctxall_mean, euc_ctxall_mean, man_ctxall_mean = None, None ,None
      cos_ctxall_max, euc_ctxall_max, man_ctxall_max = None, None ,None

    # Contexto de todas as palavras menos as stopswords
    if contextos_documentos[1] != None:
      # Recupera as medidas dos embeddings das palavras usando as estratégias MEAN e MAX
      cos_ctxclean_mean, euc_ctxclean_mean, man_ctxclean_mean = getMedidasEmbedding(lista_embeddings_mean[i],
                                                                              contextos_documentos[1])
      cos_ctxclean_max, euc_ctxclean_max, man_ctxclean_max = getMedidasEmbedding(lista_embeddings_max[i],
                                                                           contextos_documentos[1])
    else:
      cos_ctxclean_mean, euc_ctxclean_mean, man_ctxclean_mean = None, None ,None
      cos_ctxclean_max, euc_ctxclean_max, man_ctxclean_max = None, None ,None

    # Contexto de todas as palavras verbos
    if contextos_documentos[2] != None:
      # Recupera as medidas dos embeddings das palavras usando as estratégias MEAN e MAX
      cos_ctxverb_mean, euc_ctxverb_mean, man_ctxverb_mean = getMedidasEmbedding(lista_embeddings_mean[i],
                                                                           contextos_documentos[2])
      cos_ctxverb_max, euc_ctxverb_max, man_ctxverb_max = getMedidasEmbedding(lista_embeddings_max[i],
                                                                        contextos_documentos[2])
    else:
      cos_ctxverb_mean, euc_ctxverb_mean, man_ctxverb_mean = None, None ,None
      cos_ctxverb_max, euc_ctxverb_max, man_ctxverb_max = None, None ,None

    # Contexto de todas as palavras substantivos
    if contextos_documentos[3] != None:
      # Recupera as medidas dos embeddings das palavras usando as estratégias MEAN e MAX
      cos_ctxnoun_mean, euc_ctxnoun_mean, man_ctxnoun_mean = getMedidasEmbedding(lista_embeddings_mean[i],
                                                               contextos_documentos[3])
      cos_ctxnoun_max, euc_ctxnoun_max, man_ctxnoun_max = getMedidasEmbedding(lista_embeddings_max[i],
                                                               contextos_documentos[3])
    else:
      cos_ctxnoun_mean, euc_ctxnoun_mean, man_ctxnoun_mean = None, None ,None
      cos_ctxnoun_max, euc_ctxnoun_max, man_ctxnoun_max = None, None ,None

    # Contexto de todas as palavras verbos e substantivos
    if contextos_documentos[4] != None:
      # Recupera as medidas dos embeddings das palavras usando as estratégias MEAN e MAX
      cos_ctxverbnoun_mean, euc_ctxverbnoun_mean, man_ctxverbnoun_mean = getMedidasEmbedding(lista_embeddings_mean[i],
                                                                           contextos_documentos[4])
      cos_ctxverbnoun_max, euc_ctxverbnoun_max, man_ctxverbnoun_max = getMedidasEmbedding(lista_embeddings_max[i],
                                                                           contextos_documentos[4])
    else:
      cos_ctxverbnoun_mean, euc_ctxverbnoun_mean, man_ctxverbnoun_mean = None, None ,None
      cos_ctxverbnoun_max, euc_ctxverbnoun_max, man_ctxverbnoun_max = None, None ,None

    comparacao = [id_documento, indexdocumento, index_sentenca, i, str(wi), posi,
                  cos_ctxall_mean,        euc_ctxall_mean,      man_ctxall_mean,
                  cos_ctxclean_mean,      euc_ctxclean_mean,    man_ctxclean_mean,
                  cos_ctxverb_mean,       euc_ctxverb_mean,     man_ctxverb_mean,
                  cos_ctxnoun_mean,       euc_ctxnoun_mean,     man_ctxnoun_mean,
                  cos_ctxverbnoun_mean,   euc_ctxverbnoun_mean, man_ctxverbnoun_mean,
                  cos_ctxall_max,         euc_ctxall_max,       man_ctxall_max,
                  cos_ctxclean_max,       euc_ctxclean_max,     man_ctxclean_max,
                  cos_ctxverb_max,        euc_ctxverb_max,      man_ctxverb_max,
                  cos_ctxnoun_max,        euc_ctxnoun_max,      man_ctxnoun_max,
                  cos_ctxverbnoun_max,    euc_ctxverbnoun_max,  man_ctxverbnoun_max
                  ]

    lista_comparacao.append(comparacao)

    #print(comparacao)
    #print("Compara :", i)
    #print("Compara :", wi, " com o contexto")
    #print("     cos_ctxall_mean:", cos_ctxall_mean)
    #print("     euc_ctxall_mean:", euc_ctxall_mean)
    #print("     man_ctxall_mean:", man_ctxall_mean)

  del lista_tokens
  del lista_postagging
  del lista_embeddings_mean
  del lista_embeddings_max

  return lista_comparacao

### 5.2.6 Realiza a comparação do contexto com as palavras

In [ ]:
# Import das bibliotecas
from tqdm.notebook import tqdm as tqdm_notebook

logging.info("Processando {} documentos originais e perturbados.".format(len(lista_documentos_agrupados)))

# Guarda a comparacação das sentenças
resultado_comparacao = []

# Conta sentenças comparadas e não comparadas
conta_sentenca = 0

# Limpa o buffer antes de realizar a comparação
limpaBufferEmbedding()

# Barra de progresso dos documentos
lista_documentos_bar = tqdm_notebook(lista_documentos_agrupados.iterrows(), desc=f"Documentos", unit=f" documento", total=len(lista_documentos_agrupados))

# Percorre os documentos
for i, linha_documento in lista_documentos_bar:
  #if i < 10:
    #print("linha_documento:",linha_documento)
    # Recupera o id do documento
    id_documento = linha_documento[0]
    #print("id_documento:",id_documento)

    # Carrega as listas das sentenças e postagging dos documento
    lista_sentenca_documento = linha_documento[1]
    # print("lista_sentenca_documento:",lista_sentenca_documento)
    # print("len(lista_sentenca_documento):",len(lista_sentenca_documento))

    # Recupera o documento
    documento = linha_documento[2]
    #print("documento:",documento)
    # Recupera a classe documento (1-original 0-perturbado)
    #classe = linha_documento[3]
    #print("classe:",classe)

    # Localiza a POSTagging do documento agrupado
    lista_pos_documento = lista_documentos_agrupados_pos_indexado.loc[id_documento][0]
    #print("lista_pos_documento:",lista_pos_documento)
    #print("len(lista_pos_documento):",len(lista_pos_documento))

    # Troca o documento por uma versão da concatenação das palavras geradas pelo spaCy
    # Percorre a lista_pos concatenando a posição 0 dos tokens
    documento_concatenado = " ".join(concatenaListas(lista_pos_documento, pos=0))
    # print("documento_concatenado:", documento_concatenado)
    documento = documento_concatenado

    # Concatena os tokens das sentenças do documento que estão em listas separadas
    token_documento = concatenaListas(lista_pos_documento,0)

    # Concatena as pos das palavras do documento que estão em listas separadas
    pos_documento = concatenaListas(lista_pos_documento,1)

    # Gera os embeddings do documento utiliza a concatenação das 4 últimas camadas
    # Somente para os documentos originais
    if "_pert_" not in id_documento:
      embedding_documento, token_BERT_documento = getEmbeddingsDocumentoBuffer(documento, model, tokenizer)
    else:
      embedding_documento, token_BERT_documento = getEmbeddingsDocumento(documento, model, tokenizer)
    # embedding <qtde_tokens x 4096>
    #print("embedding_documento:",embedding_documento.shape)
    #print("token_BERT_documento:",token_BERT_documento)
    #print("len(token_BERT_documento):",len(token_BERT_documento))

    # Gera os embeddings da média dos embeddings de todas as palavras do documento
    embedding_documento_all_mean = getEmbeddingDocumentoMean(id_documento,
                                                             None,
                                                             embedding_documento,
                                                             token_BERT_documento,
                                                             documento,
                                                             tokenizer,
                                                             token_documento,
                                                             pos_documento,
                                                             filtro=["ALL"])
    #print("embedding_documento_all_mean:",embedding_documento_all_mean.shape)

   # Gera os embeddings da média dos embeddings de todas as palavras do documento desconsiderando stopwords
    embedding_documento_clean_mean = getEmbeddingDocumentoMean(id_documento,
                                                               None,
                                                               embedding_documento,
                                                               token_BERT_documento,
                                                               documento,
                                                               tokenizer,
                                                               token_documento,
                                                               pos_documento,
                                                               filtro=["CLEAN"])
    #print("embedding_documento_clean_mean:",embedding_documento_clean_mean.shape)

    # Gera os embeddings da média dos embeddings de todas as palavras do documento da classe VERB
    embedding_documento_verb_mean = getEmbeddingDocumentoMean(id_documento,
                                                              None,
                                                              embedding_documento,
                                                              token_BERT_documento,
                                                              documento,
                                                              tokenizer,
                                                              token_documento,
                                                              pos_documento,
                                                              filtro=["VERB","AUX"])
    #print("embedding_documento_clean_mean:",embedding_documento_verb_mean.shape)

    # Gera os embeddings da média dos embeddings de todas as palavras do documento da classe NOUN
    embedding_documento_noun_mean = getEmbeddingDocumentoMean(id_documento,
                                                              None,
                                                              embedding_documento,
                                                              token_BERT_documento,
                                                              documento,
                                                              tokenizer,
                                                              token_documento,
                                                              pos_documento,
                                                              filtro=["NOUN"])
    #print("embedding_documento_noun_mean:",embedding_documento_noun_mean.shape)

    # Gera os embeddings da média dos embeddings de todas as palavras do documento da classe VERB e NOUN
    embedding_documento_verbnoun_mean = getEmbeddingDocumentoMean(id_documento,
                                                                  None,
                                                                  embedding_documento,
                                                                  token_BERT_documento,
                                                                  documento,
                                                                  tokenizer,
                                                                  token_documento,
                                                                  pos_documento,
                                                                  filtro=["VERB","NOUN","AUX"])
    #print("embedding_documento_verbnoun_mean:",embedding_documento_verbnoun_mean.shape)

    # Agrega os embeddings dos contextos
    contextos_documentos = [embedding_documento_all_mean,
                           embedding_documento_clean_mean,
                           embedding_documento_verb_mean,
                           embedding_documento_noun_mean,
                           embedding_documento_verbnoun_mean]

    # Percorre as sentenças do documento
    for j, sentenca in enumerate(lista_sentenca_documento):
      #print("id_documento:",id_documento)
      #print("sentenca:",sentenca)

      # Carrega as POSTagging da sentença
      sentenca_token = lista_pos_documento[j][0]
      sentenca_postagging = lista_pos_documento[j][1]
      sentenca_verbos = lista_pos_documento[j][2]

      #print("sentenca_token:",sentenca_token)
      #print("len(sentenca_token):",len(sentenca_token))

      #print("sentenca_postagging:",sentenca_postagging)
      #print("len(sentenca_postagging):",len(sentenca_postagging))

      #print("sentenca_verbos:",sentenca_verbos)
      #print("len(sentenca_verbos):",len(sentenca_verbos))

      # Conta o número de sentenças com palavras comparadas
      conta_sentenca = conta_sentenca + 1

      # Recupera as medidas entre os contextos e as palavras
      lista_comparacao = comparaContextoPalavrasTodas(id_documento,
                                                      i, #index do documento
                                                      j, #index da sentença do documento i
                                                      embedding_documento,
                                                      token_BERT_documento,
                                                      sentenca,
                                                      tokenizer,
                                                      sentenca_token,
                                                      sentenca_postagging,
                                                      contextos_documentos)

      # Guarda o resultado da comparação
      resultado_comparacao = resultado_comparacao + lista_comparacao

In [ ]:
logging.info("Número de sentenças com palavras comparadas: {}.".format(conta_sentenca))
logging.info("Número de comparações                      : {}.".format(len(resultado_comparacao)))

## 5.3 Gera arquivo das comparações

### 5.3.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_COMPARACAO_CONTEXTO = "comparacao_contexto_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".csv"
NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO = "comparacao_contexto_p" + str(model_args.documentos_perturbados) + "_k" + str(model_args.top_k_predicao) + ".zip"

### 5.3.2 Gera arquivo comparação

In [ ]:
# Importa das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
resultado_comparacao = pd.DataFrame(resultado_comparacao, columns = ["id",
                                                                       "index_documento",
                                                                       "index_sentenca",
                                                                       "index_wi",
                                                                       "wi",
                                                                       "pos_i",
                                                                       "cos_ctxall_mean",       "euc_ctxall_mean",        "man_ctxall_mean",
                                                                       "cos_ctxclean_mean",     "euc_ctxclean_mean",      "man_ctxclean_mean",
                                                                       "cos_ctxverb_mean",      "euc_ctxverb_mean",       "man_ctxverb_mean",
                                                                       "cos_ctxnoun_mean",      "euc_ctxnoun_mean",       "man_ctxnoun_mean",
                                                                       "cos_ctxverbnoun_mean",  "euc_ctxverbnoun_mean",   "man_ctxverbnoun_mean",

                                                                       "cos_ctxall_max",        "euc_ctxall_max",         "man_ctxall_max",
                                                                       "cos_ctxclean_max",      "euc_ctxclean_max",       "man_ctxclean_max",
                                                                       "cos_ctxverb_max",       "euc_ctxverb_max",        "man_ctxverb_max",
                                                                       "cos_ctxnoun_max",       "euc_ctxnoun_max",        "man_ctxnoun_max",
                                                                       "cos_ctxverbnoun_max",   "euc_ctxverbnoun_max",    "man_ctxverbnoun_max"
                                                                       ])

# Nome do arquivo original
nome_arquivo = DIRETORIO_LOCAL + NOME_ARQUIVO_COMPARACAO_CONTEXTO

# Salva o arquivo original
resultado_comparacao.to_csv(nome_arquivo,  sep=";", index=False)

### 5.3.3 Carrega os dados

Carrega os dados das sentencas a partir dos arquivos.


In [ ]:
# Importa das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
resultado_comparacao = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_COMPARACAO_CONTEXTO, sep=";", encoding="UTF-8")

logging.info("Quantidade de comparações: {}.".format(len(resultado_comparacao)))

In [ ]:
resultado_comparacao.sample(5)

### 5.3.4 Compacta e copia o arquivo para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

Compacta o arquivo.

Usa o zip para compactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens

In [ ]:
!zip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO"

Copia o arquivo  para o GoogleDrive

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

    # Copia o arquivo das comparações para o google drive
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_COMPARACAO_CONTEXTO_COMPACTADO" "$DIRETORIO_DRIVE"

    logging.info("Terminei a cópia do arquivo.")

# Tempo final de processamento

100 = Tempo processamento:  16:14:32 (h:mm:ss)

In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))